In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

import bp

In [2]:
U = bp.variableNode("Unblocked FEV1", 2, 6, 2)
B = bp.variableNode("Small airway blockage", 0, 1, 0.5)
FEV1 = bp.variableNode("FEV1", 0, 6, 3)

In [3]:
graph = BayesianNetwork([(U.name, FEV1.name), (B.name, FEV1.name)])

cpt_fev1 = TabularCPD(
    variable=FEV1.name,
    variable_card=len(FEV1.bins)-1,
    values=bp.calc_pgmpy_cpt(U, B, FEV1).tolist(),
    evidence=[U.name, B.name],
    evidence_card=[len(U.bins)-1, len(B.bins)-1],
)

marginal_b = TabularCPD(
    variable=B.name,
    variable_card=len(B.bins)-1,
    values=B.marginal(B),
    evidence=[],
    evidence_card=[],
)

marginal_u = TabularCPD(
    variable=U.name,
    variable_card=len(U.bins)-1,
    values=U.marginal(U),
    evidence=[],
    evidence_card=[],
)

graph.add_cpds(cpt_fev1, marginal_b, marginal_u)

graph.check_model()


True